In [1]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Embedding, LSTM, Dense, Concatenate
from tensorflow.keras.utils import plot_model

# 시드 설정 및 데이터 생성
np.random.seed(2020)
hamburger_img = np.random.random((1000, 28, 28, 1)).astype(np.float32)
customer_form = np.random.randint(10000, size=(1000, 100)).astype(np.int32)
hamburger_rate = np.round(np.random.random((1000,)) * 5, 1).astype(np.float32)
update_for_hamburger = np.random.randint(50, size=(1000,)).astype(np.int32)

In [2]:
# 모델의 입력 정의
img_input = Input(shape=(28, 28, 1), name='hamburger_img')
form_input = Input(shape=(100,), dtype='int32', name='customer_form')

# 햄버거 이미지 처리
x_1 = Conv2D(32, (3, 3), activation='relu')(img_input)
x_1 = Conv2D(32, (3, 3), activation='relu')(x_1)
x_1 = MaxPooling2D(strides=2)(x_1)
x_1 = GlobalAveragePooling2D()(x_1)

# 햄버거 평가 처리
x_2 = Embedding(input_dim=10000, output_dim=64, input_length=100, mask_zero=True)(form_input)
x_2 = LSTM(128)(x_2)

# Concatenate로 합치기
x = Concatenate()([x_1, x_2])

C:\Users\human\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [3]:
# 출력 레이어 정의
rate_pred = Dense(1, name='hamburger_rate')(x)
update_pred = Dense(50, activation='softmax', name='update_for_hamburger')(x)

# 모델 정의
model = Model(inputs=[img_input, form_input], outputs=[rate_pred, update_pred])

# 모델 시각화 및 요약 출력
plot_model(model, show_shapes=True)
model.summary()

# 모델 컴파일
model.compile(
    optimizer='adam',
    loss={'hamburger_rate': 'mse', 'update_for_hamburger': 'sparse_categorical_crossentropy'},
    metrics={'hamburger_rate': 'mse', 'update_for_hamburger': 'accuracy'}
)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ hamburger_img       │ (None, 28, 28, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 26, 26,    │        320 │ hamburger_img[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 24, 24,    │      9,248 │ conv2d[0][0]      │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ customer_form       │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 12, 12,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 100, 64)   │    640,000 │ customer_form[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 100)       │          0 │ customer_form[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 32)        │          0 │ max_pooling2d[0]… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 128)       │     98,816 │ embedding[0][0],  │
│                     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 160)       │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ hamburger_rate      │ (None, 1)         │        161 │ concatenate[0][0] │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ update_for_hamburg… │ (None, 50)        │      8,050 │ concatenate[0][0] │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 756,595 (2.89 MB)

 Trainable params: 756,595 (2.89 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
# 모델 학습 (리스트 형태의 입력 사용)
history = model.fit(
    [hamburger_img, customer_form],    # 리스트 형태의 입력
    [hamburger_rate, update_for_hamburger],  # 리스트 형태의 출력
    epochs=2,
    batch_size=32
)


Epoch 1/2


C:\Users\human\AppData\Roaming\Python\Python312\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['hamburger_img', 'customer_form']. Received: the structure of inputs=('*', '*')
  warnings.warn(


32/32 ━━━━━━━━━━━━━━━━━━━━ 6s 60ms/step - hamburger_rate_loss: 5.8917 - hamburger_rate_mse: 5.8942 - loss: 9.8222 - update_for_hamburger_accuracy: 0.0180 - update_for_hamburger_loss: 3.9280
Epoch 2/2
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - hamburger_rate_loss: 1.8379 - hamburger_rate_mse: 1.8386 - loss: 5.7444 - update_for_hamburger_accuracy: 0.0235 - update_for_hamburger_loss: 3.9059
